In [356]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import gensim
import spacy
import texthero as hero


In [325]:
data1_src = "data1/News _dataset/"


In [326]:
data1_fake ="Fake.csv"
df_fake = pd.read_csv(data1_src + data1_fake)
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [327]:
df_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [328]:
df_fake.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [329]:
df_fake.groupby(['subject']).count()

,title,text,date
subject,,,
Government News,1570,1570,1570
Middle-east,778,778,778
News,9050,9050,9050
US_News,783,783,783
left-news,4459,4459,4459
politics,6841,6841,6841


The above caegories, except for 'Middle-east' news, may have overlapping / similar news stories. Hencetwo categories will be selected to match a similar category in true news data.

In [330]:
# df_train[(df_train["fold"]==1) | (df_train["fold"]==2)]
fnews = (df_fake['subject']=='politics' )| (df_fake['subject'] == 'US_News')

In [331]:
fdf = df_fake[fnews]

In [332]:
fdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 9050 to 22702
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    7624 non-null   object
 1   text     7624 non-null   object
 2   subject  7624 non-null   object
 3   date     7624 non-null   object
dtypes: object(4)
memory usage: 297.8+ KB


Only news stories are going to be used to keep the data uniform.Tweets have shorter legth than stories.

In [333]:
ftweets = df_fake['title'].str.contains("(TWEETS)", case = False) == False

/usr/local/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [334]:
f_df = fdf[ftweets]


<ipython-input-334-94b778f00658>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  f_df = fdf[ftweets]


In [335]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7585 entries, 9050 to 22702
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    7585 non-null   object
 1   text     7585 non-null   object
 2   subject  7585 non-null   object
 3   date     7585 non-null   object
dtypes: object(4)
memory usage: 296.3+ KB


There do not seem to be a number of tweets in the dataset. 

In [336]:
f_df['target'] = 0

<ipython-input-336-bae77650778f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_df['target'] = 0


In [273]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7585 entries, 9050 to 22702
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    7585 non-null   object
 1   text     7585 non-null   object
 2   subject  7585 non-null   object
 3   date     7585 non-null   object
 4   target   7585 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 355.5+ KB


In [344]:
f_df = f_df[['text', 'target']]
## 
f_df = f_df.reset_index(drop = True)

In [345]:
f_df.head()

,text,target
0,"According to The Hill, Democrat Senator Bob Ca...",0
1,If we protect every other government building ...,0
2,Ami Horowitz is fantastic! Check out this man ...,0
3,Just one more reminder of why President Trump ...,0
4,Thank goodnesss Jeff Sessions is moving on fin...,0


That marks the basic extrtaction of "fake news" dataset. It has to be processed further.

Now, it i stime to take a look at the True dataset.

In [370]:
data1_true = "True.csv"
df_true = pd.read_csv(data1_src + data1_true)
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [371]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [372]:
df_true.describe()

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [373]:
df_true.groupby(['subject']).count()

,title,text,date
subject,,,
politicsNews,11272,11272,11272
worldnews,10145,10145,10145


Unlike the fake news dataset, true news is gathered only from Reuters and has only two categories. It would be a good idea to consider only US political news in order to look as homogenous as possible with the fake news data, in terms of the topics.

In [374]:
tnews = df_true['subject'] == 'politicsNews'
truenews_df = df_true[tnews]
truenews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11272 entries, 0 to 11271
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    11272 non-null  object
 1   text     11272 non-null  object
 2   subject  11272 non-null  object
 3   date     11272 non-null  object
dtypes: object(4)
memory usage: 440.3+ KB


In [375]:
truenews_df.describe()

,title,text,subject,date
count,11272,11272,11272,11272
unique,11150,11214,1,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"November 9, 2016"
freq,14,8,11272,115


When Reuters posts are checked it will have a signature mark of "(Reuters)" at the beginning of the news story.

In [376]:
tdf = truenews_df['text'].str.contains("(Reuters)",case = False)==True

/usr/local/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [377]:
t_df = truenews_df[tdf]

In [378]:
t_df['target'] = 1

<ipython-input-378-b637b80800ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df['target'] = 1


In [379]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11239 entries, 0 to 11271
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    11239 non-null  object
 1   text     11239 non-null  object
 2   subject  11239 non-null  object
 3   date     11239 non-null  object
 4   target   11239 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 526.8+ KB


In [380]:
t_df = t_df[['text', 'target']]
t_df = t_df.reset_index(drop= True)


To-do: yymm and a count for that yymm would be better to plot as a line plot along with the true data. A wordnet like graph would be useful too.

It is time to pre process both the data frames. since we are only interested in the text and target.
The two data sets are not the same sixe, true set has about 0% more than the fake set. 
shall I reindex the dataframes?


In order to install Texthero for cleaning, the following command had to be used:
!pip3 install gensim==3.8.3

In [381]:
# let us try text hero
t_df['text'] = hero.clean(t_df['text'])

/usr/local/lib/python3.9/site-packages/texthero/preprocessing.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pattern, symbols)
/usr/local/lib/python3.9/site-packages/texthero/preprocessing.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(rf"([{string.punctuation}])+", symbol)


In [382]:
t_df.head()

,text,target
0,washington reuters head conservative republica...,1
1,washington reuters transgender people allowed ...,1
2,washington reuters special counsel investigati...,1
3,washington reuters trump campaign adviser geor...,1
4,seattle washington reuters president donald tr...,1


The signature of Reuters can be taken out.

In [383]:
t_df['text'] = t_df['text'].map(lambda x: x.split('reuters',1)[1])

In [384]:
t_df.head()

,text,target
0,head conservative republican faction u congre...,1
1,transgender people allowed first time enlist ...,1
2,special counsel investigation links russia pr...,1
3,trump campaign adviser george papadopoulos to...,1
4,president donald trump called u postal servic...,1


That marks the extraction of "true news"dataset.
Now, off to turn text to words, vectors and models.

In [ ]:
Both the datsets be merged into one but randomly.

In [395]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11239 entries, 0 to 11238
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11239 non-null  object
 1   target  11239 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 175.7+ KB


In [396]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7585 entries, 0 to 7584
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7585 non-null   object
 1   target  7585 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 118.6+ KB


In [ ]:
dfs = [df1, df2, df3, df4]
random.shuffle(dfs)
pd.concat(dfs, axis=1)

In [387]:
import random


In [390]:
t_df.head()

,text,target
0,head conservative republican faction u congre...,1
1,transgender people allowed first time enlist ...,1
2,special counsel investigation links russia pr...,1
3,trump campaign adviser george papadopoulos to...,1
4,president donald trump called u postal servic...,1


In [391]:
f_df.head()

,text,target
0,"According to The Hill, Democrat Senator Bob Ca...",0
1,If we protect every other government building ...,0
2,Ami Horowitz is fantastic! Check out this man ...,0
3,Just one more reminder of why President Trump ...,0
4,Thank goodnesss Jeff Sessions is moving on fin...,0


In [ ]:
result = pd.concat([list of DataFrames], axis=0, join='outer', ignore_index=False)

In [411]:
tf_df = pd.concat([t_df,f_df], axis = 0, ignore_index = True)

In [412]:
tf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18824 entries, 0 to 18823
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18824 non-null  object
 1   target  18824 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 294.2+ KB


In [413]:
tf_df.head(-5)

,text,target
0,head conservative republican faction u congre...,1
1,transgender people allowed first time enlist ...,1
2,special counsel investigation links russia pr...,1
3,trump campaign adviser george papadopoulos to...,1
4,president donald trump called u postal servic...,1
...,...,...
18814,21st Century Wire says As 21WIRE reported earl...,0
18815,21st Century Wire says It s a familiar theme. ...,0
18816,Patrick Henningsen 21st Century WireRemember ...,0
18817,21st Century Wire says Al Jazeera America will...,0


In [414]:
tf_df = tf_df.sample(frac = 1)

In [415]:
tf_df.head(-5)

,text,target
3946,u senate voted tuesday confirm attorney jay c...,1
3541,"u president donald trump' time office ""comple...",1
11687,President Trump received a warm welcome when h...,0
2772,u agency could bring civil charges donald tru...,1
8011,mexicans favor u democratic presidential cand...,1
...,...,...
18316,Join Patrick every Wednesday at Independent T...,0
1655,u senate republicans reached tentative budget...,1
11741,How should the gay community feel about the Tr...,0
5411,u vice president mike pence heads europe week...,1


In [416]:
tf_df.reset_index()

,index,text,target
0,3946,u senate voted tuesday confirm attorney jay c...,1
1,3541,"u president donald trump' time office ""comple...",1
2,11687,President Trump received a warm welcome when h...,0
3,2772,u agency could bring civil charges donald tru...,1
4,8011,mexicans favor u democratic presidential cand...,1
...,...,...,...
18819,10576,use nevertrump hashtag grew twitter monday de...,1
18820,9747,maine state official wednesday validated peti...,1
18821,14970,"University of Wisconsin Madison, where NOT to ...",0
18822,5197,u president donald trump tried win skeptics c...,1


Well, sampling gave 5 extra rows. Is there a better way to shufflel the rows
Also, there is s preceding 'u'for some of the true news datsets.


train/test split

In [418]:
import numpy as np
from sklearn.model_selection import train_test_split

In [420]:
X = tf_df[['text']]
y = tf_df[['target']]


In [421]:
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

In [422]:
X_train.head()

,text
15383,How very progressive While Hillary insults wom...
17326,Boehner s hightailing it out of DC just in tim...
17657,Disney s just doing their part to advance a ra...
150,u republican senator mike lee decided whether...
14044,https://www.youtube.com/watch?v=RRPSCqkAJgk


In [423]:
y_train.head()

,target
15383,0
17326,0
17657,0
150,1
14044,0


from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text
card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df.clean_text)]
#display the tagged docs
card_docs

In [424]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.text)
X_train_counts.shape

(12612, 61658)

In [425]:
count_vect.vocabulary_.get(u'algorithm')

4357

In [426]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(12612, 61658)

In [428]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(12612, 61658)

In [429]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train.target)

In [438]:
X_test_counts = count_vect.transform(X_test.text)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

predicted = clf.predict(X_test_tfidf)

In [439]:
np.mean(predicted == y_test.target)


0.9755312298776562

In [440]:
from sklearn import metrics
print(metrics.classification_report(y_test.target, predicted))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97      2526
           1       0.96      1.00      0.98      3686

    accuracy                           0.98      6212
   macro avg       0.98      0.97      0.97      6212
weighted avg       0.98      0.98      0.98      6212



In [441]:
metrics.confusion_matrix(y_test.target, predicted)

array([[2374,  152],
       [   0, 3686]])

It is not ablle to clearly identify some fake news items yet, just like humans. 